<a href="https://colab.research.google.com/github/TheMatheusLs/ChessWithPython2.0/blob/master/1_Teste_de_hiperpar%C3%A2metros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação dos Dados

In [ ]:
import numpy as np
import pandas as pd



In [ ]:
n_slots = 128 # Número de slots do espectro elástico

# Cria o nome para as colunas
columns = ["nReq"]
columns.extend([f"in_{i}" for i in range(n_slots)])
columns.extend(["OUT"])
columns.extend([f"out_{i}" for i in range(n_slots)])

# Cria um dataframe com os nomes das colunas
df = pd.read_csv("rn_database.csv", delimiter =";", names=columns, index_col=False)

df.shape

(946789, 258)

In [ ]:
df.shape

(946789, 258)

In [ ]:
# Separa os dados de entrada em uma array do numpy
inC = ["nReq"]
inC.extend([f"in_{i}" for i in range(n_slots)])
input = df[inC].to_numpy()
input.shape

(946789, 129)

In [ ]:
# Separa os dados de saída bruta em uma array do numpy
outC = [f"out_{i}" for i in range(n_slots)]
output = df[outC].to_numpy()
output.shape

(946789, 128)

Entrada:

In [ ]:
input[1000]

array([3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 3, 2, 2, 2,
       2, 0, 0, 2, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1,
       2, 2, 1, 1, 1, 1, 1, 3, 3, 3, 2, 2, 2, 2, 2, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
      dtype=int64)

Saída Original:

In [ ]:
output[1000]

array([3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 3, 2, 2, 2, 2,
       0, 0, 2, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2,
       2, 1, 1, 1, 1, 1, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], dtype=int64)

Realiza o procedimento de limpeza dos dados de saída.

Subtrai a saída pela entrada, assim, o dados de saída ficarão em binário

In [ ]:
# Clean output

aux = []

for i in range(input.shape[0]):
    aux.append(output[i] - input[i][1::])
outputClean = np.array(aux)

Saída após o processamento:

In [ ]:
outputClean[1000]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

# Preparação da Rede Neural

In [ ]:
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Rodando na {device}")

class NetBinarySimple(nn.Module):
    def __init__(self):
        super(NetBinarySimple, self).__init__()
        self.fc1 = nn.Linear(129, 256)
        self.fc2 = nn.Linear(256, 128)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

Rodando na cpu


In [ ]:
import torch
print(torch.cuda.is_available())

False


In [ ]:
NetBinarySimpleModel = NetBinarySimple().to(device)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class MyDatasetFF(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = torch.from_numpy(inputs).float()
        self.labels = torch.from_numpy(labels).float()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]
    
# Create a dataset and a dataloader
datasetFF = MyDatasetFF(input, outputClean)

dataloader = DataLoader(datasetFF, batch_size=32, shuffle=True)

# Define a loss function and an optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(NetBinarySimpleModel.parameters())

In [ ]:
# Train the model
for epoch in range(20):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = NetBinarySimpleModel(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.0499420166015625
Epoch 2, Loss: 0.034125588834285736
Epoch 3, Loss: 0.015199622139334679
Epoch 4, Loss: 0.01624082587659359
Epoch 5, Loss: 0.008938159793615341
Epoch 6, Loss: 0.011499809101223946
Epoch 7, Loss: 0.01095719262957573
Epoch 8, Loss: 0.009315535426139832
Epoch 9, Loss: 0.030405286699533463
Epoch 10, Loss: 0.01123095117509365
Epoch 11, Loss: 0.016398005187511444
Epoch 12, Loss: 0.03483835980296135
Epoch 13, Loss: 0.02133520506322384
Epoch 14, Loss: 0.029620125889778137
Epoch 15, Loss: 0.011968830600380898
Epoch 16, Loss: 0.019264955073595047
Epoch 17, Loss: 0.00269039417617023
Epoch 18, Loss: 0.011167623102664948
Epoch 19, Loss: 0.043889328837394714
Epoch 20, Loss: 0.006043624132871628


## Visualização do retorno do modelo

Entrada dos dados

In [ ]:
req = 50
t1 = input[req]
t1

array([2, 2, 2, 1, 1, 1, 0, 2, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

Saída esperada dos dados

In [ ]:
outputClean[req]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

Saída da rede neural após o tratamento

In [ ]:
s2 = np.round(NetBinarySimpleModel(torch.Tensor(t1)).detach().numpy())
s2

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

Subtraindo a saída da rede neural com o saída esperada podemos observar a ocorrência do número de erros por slots. Como após os subtrações os valores podem assumir -1, 0 e 1, faremos o módulo dele. Visto que -1 e 1 são erros.

In [ ]:
qtdErr = np.sum(np.abs(s2 - outputClean[req]))
qtdErr

0.0

Ou seja, para a requisição acima, não houveram erros.

Realizando a análise para todas as requisições. Se separando os dados em número de ocorrência de erros, teremos:

In [ ]:
numReqs = input.shape[0]
arrErr = np.zeros(numReqs)
arrErr

for r in range(numReqs):
    inp = input[r]
    out = np.round(NetBinarySimpleModel(torch.Tensor(inp)).detach().numpy())

    err = np.sum(np.abs(outputClean[r] - out))

    arrErr[r] = err

arrErr

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
import collections

dict(collections.Counter(arrErr))

{0.0: 713947,
 1.0: 132110,
 2.0: 51480,
 3.0: 24186,
 4.0: 11625,
 6.0: 3143,
 5.0: 5888,
 7.0: 1630,
 8.0: 1024,
 9.0: 684,
 11.0: 315,
 10.0: 457,
 12.0: 219,
 13.0: 48,
 14.0: 16,
 15.0: 8,
 16.0: 6,
 18.0: 2,
 17.0: 1}

In [ ]:
errPercent = (numReqs - _[0.0]) / numReqs
errPercent

0.24592807901232483

Ou seja, após o treinamento (sem validação cruzada) de 20 épocas houverão 24% de requisições erradas neste conjunto de dados.

Vamos treinar mais 80 épocas para completar 100 e analisar se os resultados evoluem.

In [ ]:
# Train the model
for epoch in range(20):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = NetBinarySimpleModel(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch +21}, Loss: {loss.item()}")

Epoch 21, Loss: 0.0008457716321572661
Epoch 22, Loss: 0.01574346050620079
Epoch 23, Loss: 0.00701555609703064
Epoch 24, Loss: 0.0006948796799406409
Epoch 25, Loss: 0.0072281197644770145
Epoch 26, Loss: 0.020708926022052765
Epoch 27, Loss: 0.012293537147343159
Epoch 28, Loss: 0.010215168818831444
Epoch 29, Loss: 0.004975388757884502
Epoch 30, Loss: 0.010087254457175732
Epoch 31, Loss: 0.0025553246960043907
Epoch 32, Loss: 0.008060634136199951
Epoch 33, Loss: 0.0031817734707146883
Epoch 34, Loss: 0.013920003548264503
Epoch 35, Loss: 0.010460039600729942
Epoch 36, Loss: 0.00454371701925993
Epoch 37, Loss: 0.005126320756971836
Epoch 38, Loss: 0.005375952459871769
Epoch 39, Loss: 0.017175432294607162
Epoch 40, Loss: 0.013311115093529224


Verificando a ocorrência de erros novamente

In [ ]:
numReqs = input.shape[0]
arrErr2 = np.zeros(numReqs)
arrErr2

for r in range(numReqs):
    inp = input[r]
    out = np.round(NetBinarySimpleModel(torch.Tensor(inp)).detach().numpy())

    err = np.sum(np.abs(outputClean[r] - out))

    arrErr2[r] = err

arrErr2

array([0., 0., 0., ..., 2., 0., 0.])

In [ ]:
d2 = dict(collections.Counter(arrErr2))
d2 

{0.0: 755916,
 2.0: 44162,
 1.0: 112025,
 3.0: 18229,
 4.0: 7865,
 6.0: 2214,
 5.0: 3879,
 8.0: 587,
 7.0: 891,
 9.0: 406,
 11.0: 190,
 10.0: 299,
 12.0: 106,
 13.0: 12,
 14.0: 6,
 15.0: 1,
 16.0: 1}

In [ ]:
errPercent = (numReqs - d2[0.0]) / numReqs
errPercent

0.20160035657363995

Não houve variação significativa no resultado

In [ ]:
# Train the model
for epoch in range(60):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = NetBinarySimpleModel(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch +41}, Loss: {loss.item()}")

Epoch 41, Loss: 0.005657064262777567
Epoch 42, Loss: 0.010488779284060001
Epoch 43, Loss: 0.008289593271911144
Epoch 44, Loss: 0.0028080942574888468
Epoch 45, Loss: 0.012593844905495644
Epoch 46, Loss: 0.009315289556980133
Epoch 47, Loss: 0.031193461269140244
Epoch 48, Loss: 0.008986910805106163
Epoch 49, Loss: 0.011592714115977287
Epoch 50, Loss: 0.002814812818542123
Epoch 51, Loss: 0.00020223479077685624
Epoch 52, Loss: 0.004518668167293072
Epoch 53, Loss: 0.008927787654101849
Epoch 54, Loss: 0.0010110127041116357
Epoch 55, Loss: 0.02909809909760952
Epoch 56, Loss: 0.0035059438087046146
Epoch 57, Loss: 0.027458157390356064
Epoch 58, Loss: 0.0033214292488992214
Epoch 59, Loss: 0.0015481984009966254
Epoch 60, Loss: 0.005059326998889446
Epoch 61, Loss: 0.006688252091407776
Epoch 62, Loss: 0.0028999364003539085
Epoch 63, Loss: 0.010339212603867054
Epoch 64, Loss: 0.0010419224854558706
Epoch 65, Loss: 0.02473708800971508
Epoch 66, Loss: 0.030963856726884842
Epoch 67, Loss: 0.0042826095595

In [ ]:
numReqs = input.shape[0]
arrErr3 = np.zeros(numReqs)
arrErr3

for r in range(numReqs):
    inp = input[r]
    out = np.round(NetBinarySimpleModel(torch.Tensor(inp)).detach().numpy())

    err = np.sum(np.abs(outputClean[r] - out))

    arrErr3[r] = err

arrErr3

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
d3 = dict(collections.Counter(arrErr3))
d3

{0.0: 827322,
 1.0: 75234,
 2.0: 24715,
 3.0: 10639,
 7.0: 477,
 6.0: 1040,
 4.0: 4555,
 5.0: 1966,
 9.0: 219,
 8.0: 287,
 11.0: 111,
 13.0: 6,
 12.0: 60,
 10.0: 155,
 14.0: 2,
 15.0: 1}

In [ ]:
errPercent = (numReqs - d3[0.0]) / numReqs
errPercent

0.1261812293974687

Houve uma boa evolução ao final de 100 épocas. Espera-se que com mais treinamento (de preferência cruzado) o modelo evolua melhor. 

É importante investigar os resultados para observar se esse montante de erros estão acontecendo no primeiro slots de alocação, visto ele ser o mais importante do espectro de saída.

Essa análise vai trazer uma visão da aplicação do modelo em substição da heurística.

In [ ]:
# Save the model
torch.save({
            'epoch': 100,
            'model_state_dict': NetBinarySimpleModel.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': criterion,
            }, 'model_treined2.pth')
